In [4]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

In [5]:
data = pd.read_excel('online_retail_II.xlsx', sheet_name='Year 2010-2011')

In [6]:
data.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [7]:
print(data.isnull().sum())

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64


Prepare Data

In [8]:
data.dropna(inplace=True)
data = data[~data['Invoice'].str.contains('C', na=False)]  # remove cancelled orders
data = data[(data['Quantity'] > 0)]  # remove negative quantities
data = data[(data['Price'] > 0)]  # remove free items
data.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


Select Country : For homework select United Kingdom

In [9]:
uk_data = data[data['Country'] == 'United Kingdom']
uk_data.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [10]:
def create_apriori_data(dataframe, id=False):
    if id:
        grouped = dataframe.groupby(['Invoice', 'StockCode'], as_index=False).agg({'Quantity': 'sum'})
        apriori_data = pd.pivot(data=grouped, index='Invoice', columns='StockCode', values='Quantity').fillna(0).applymap(lambda x: 1 if x > 0 else 0)
        return apriori_data
    else:
        grouped = dataframe.groupby(['Invoice', 'Description'], as_index=False).agg({'Quantity': 'sum'})
        apriori_data = pd.pivot(data=grouped, index='Invoice', columns='Description', values='Quantity').fillna(0).applymap(lambda x: 1 if x > 0 else 0)
    return apriori_data

In [11]:
uk_apriori_data = create_apriori_data(uk_data, True)
uk_apriori_data.head(5)

<ipython-input-10-60eb1d85d1e7>:4: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  apriori_data = pd.pivot(data=grouped, index='Invoice', columns='StockCode', values='Quantity').fillna(0).applymap(lambda x: 1 if x > 0 else 0)


StockCode,10002,10080,10120,10125,10133,10135,11001,15030,15034,15036,15039,16008,16010,16011,16012,16014,16015,16016,16033,16043,16045,16046,16048,16049,16052,16054,16216,16218,16219,16225,16235,16236,16237,16238,16254,16259,17001,17003,17021,17038,...,90209B,90209C,90210A,90210B,90210C,90210D,90211A,90211B,90212B,90212C,90214A,90214B,90214C,90214D,90214E,90214F,90214G,90214H,90214I,90214J,90214K,90214L,90214M,90214N,90214O,90214P,90214R,90214S,90214T,90214U,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
Invoice,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
536369,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Define Rules

In [12]:
freq_items = apriori(uk_apriori_data, min_support=0.01, use_colnames=True)
uk_rules = association_rules(freq_items, metric='support', min_threshold=0.01)

In [13]:
sorted_rules = uk_rules.sort_values(by='lift', ascending=False)
sorted_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
506,(22916),(22917),0.010753,0.010873,0.010153,0.944134,86.829038,0.010036,17.705365
507,(22917),(22916),0.010873,0.010753,0.010153,0.933702,86.829038,0.010036,14.921137
511,(23171),(23170),0.013637,0.015980,0.011534,0.845815,52.930211,0.011316,6.382074
510,(23170),(23171),0.015980,0.013637,0.011534,0.721805,52.930211,0.011316,3.545575
483,(22746),(22745),0.012556,0.015619,0.010153,0.808612,51.769856,0.009956,5.143389


Get Item Name

In [43]:
prod_code1 = 23171
prod_name1 = uk_data[uk_data['StockCode'] == prod_code1][['Description']].values[0].tolist()
print(prod_name1)

['REGENCY TEA PLATE GREEN ']


In [44]:
prod_code2 = 22916
prod_name2 = uk_data[uk_data['StockCode'] == prod_code2][['Description']].values[0].tolist()
print(prod_name2)

['HERB MARKER THYME']


In [45]:
prod_code3 = 22746
prod_name3 = uk_data[uk_data['StockCode'] == prod_code3][['Description']].values[0].tolist()
print(prod_name3)

["POPPY'S PLAYHOUSE LIVINGROOM "]


Recommend 5 products by given product id.

In [46]:
def recommend_products(product_id):
  recommended_products = []
  for i, product in sorted_rules["antecedents"].items():
    for j in list(product):
        if j == product_id:
            recommended_products.append(list(sorted_rules.iloc[i]["consequents"]))

  recommended_products = list({item for item_list in recommended_products for item in item_list}) # To get unique products
  return recommended_products[:5]

In [47]:
recommendation1 = recommend_products(prod_code1)
recommendation2 = recommend_products(prod_code2)
recommendation3 = recommend_products(prod_code3)

In [48]:
recommendation1_names = []
for i in recommendation1:
  recommendation1_names.append(uk_data[uk_data['StockCode'] == i][['Description']].values[0].tolist())

recommendation2_names = []
for i in recommendation2:
  recommendation2_names.append(uk_data[uk_data['StockCode'] == i][['Description']].values[0].tolist())

recommendation3_names = []
for i in recommendation3:
  recommendation3_names.append(uk_data[uk_data['StockCode'] == i][['Description']].values[0].tolist())

In [49]:
print(f"Target Product ID (which is in the cart): {prod_code1}\nProduct Name: {prod_name1}")
print(f"Recommended Products: {recommendation1}\nProduct Names: {recommendation1_names}")


Target Product ID (which is in the cart): 23171
Product Name: ['REGENCY TEA PLATE GREEN ']
Recommended Products: [22382, 20726]
Product Names: [['LUNCH BAG SPACEBOY DESIGN '], ['LUNCH BAG WOODLAND']]


In [50]:
print(f"Target Product ID (which is in the cart): {prod_code2}\nProduct Name: {prod_name2}")
print(f"Recommended Products: {recommendation2}\nProduct Names: {recommendation2_names}")

Target Product ID (which is in the cart): 22916
Product Name: ['HERB MARKER THYME']
Recommended Products: [22385]
Product Names: [['JUMBO BAG SPACEBOY DESIGN']]


In [51]:
print(f"Target Product ID (which is in the cart): {prod_code3}\nProduct Name: {prod_name3}")
print(f"Recommended Products: {recommendation3}\nProduct Names: {recommendation3_names}")

Target Product ID (which is in the cart): 22746
Product Name: ["POPPY'S PLAYHOUSE LIVINGROOM "]
Recommended Products: ['85099B', 23203, 22383]
Product Names: [['JUMBO BAG RED RETROSPOT'], ['JUMBO BAG DOILEY PATTERNS'], ['LUNCH BAG SUKI  DESIGN ']]
